In [14]:
import pickle
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import mean_squared_error

In [15]:
df = pickle.load(open('/Users/yangjin-i/Downloads/YogingaDataSet.pkl','rb'))

In [16]:
print(df.tail())

         Restaurant UserID                                               Menu  \
23204  원할머니보쌈-상암배달점  in**님    맛있게 매운 의성마늘떡맵쌈/1(보쌈 사이즈 선택(중（생육710g이상,수육500g）))   
23205       장미로타리크럽  jk**님                      김.두.삼（김치두부삼겹살）/1,문어 초무침과 소면/1   
23206       장미로타리크럽  ja**님                                 문어삼합/1(사이즈 선택(3人))   
23207       장미로타리크럽     손님  유자보쌈（보쌈＋보쌈김치＋부추무침）/1(사이즈 선택(반근（300g）)),문어 초무침과...   
23208       장미로타리크럽  wd**님                                   통다리 문어버터구이와 땅콩/1   

                                                  Review Total Taste Quantity  \
23204  맛과 양 모두 만족스럽습니다! 맛있게 잘 먹겠습니다~ 배달도 빠르고 포장도 너무 깔...     5     5        5   
23205  배달이 좀 걸린듯 하지만 통두부에 삼겹김치.문어소면 맛있게 잘먹어요.폭풍흡입으로 사...     4     5        5   
23206                    여러번 먹었는데 맛있어요 사진이 없네요ㅠㅠ 호다닥 먹어서     5     5        5   
23207  친구네 놀러가서 시켜봤는데 진짜 전메뉴가 다 맛있다고 하는 친구말이 맞는거 같네요!...     5     5        4   
23208  그거알아요? 여기 메뉴 다맛있다는거☆ 항상오는 서비스 홍합탕 돈주고 시켜야되는거 아...     5     5        5   

      Delivery           D

In [25]:
df.rename(columns={'Total':'Rating'},inplace = True)
df['Rating'] = pd.to_numeric(df['Rating'])

print(df_ratings.tail())
print(df_restaurant.tail())

         Restaurant UserID  Rating
23204  원할머니보쌈-상암배달점  in**님       5
23205       장미로타리크럽  jk**님       4
23206       장미로타리크럽  ja**님       5
23207       장미로타리크럽     손님       5
23208       장미로타리크럽  wd**님       5
         Restaurant Delivery Quantity
23204  원할머니보쌈-상암배달점        5        5
23205       장미로타리크럽        4        5
23206       장미로타리크럽        5        5
23207       장미로타리크럽        4        4
23208       장미로타리크럽        5        5


In [26]:
n_users = len(df['UserID'].unique())
n_restaurants = len(df['Restaurant'].unique())

n_users, n_restaurants

(1309, 635)

In [27]:
df['Rating'].describe()

count    358132.000000
mean          4.739440
std           0.731522
min           0.000000
25%           5.000000
50%           5.000000
75%           5.000000
max           5.000000
Name: Rating, dtype: float64

In [28]:
#user-restaurant utility matrix
#UserID = '손님' 제거
user_resaurant = df.pivot_table('Rating',index='UserID',columns = 'Restaurant',aggfunc='max')
user_resaurant.drop(user_resaurant.loc[['손님'],:].index, inplace = True)
print(user_resaurant.shape)

#restaurant-restaurant utility matrix
retaurant_user = df.pivot_table('Rating',index='Restaurant',columns = 'UserID',aggfunc='max')
retaurant_user.drop(retaurant_user[['손님']], inplace = True, axis = 1)
print(retaurant_user.shape)



(1308, 635)
(635, 1308)


In [29]:
#결측치 0으로 대체
retaurant_user.fillna(0,inplace = True)
user_resaurant.fillna(0,inplace = True)

print(retaurant_user.head())
print(user_resaurant.head())

UserID                   --**님  -0**님  -9**님  -_**님  -b**님  -c**님  -e**님  \
Restaurant                                                                 
                    5.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
10평파스타&화덕피자         0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
1인기사식당              0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
1인살로만강명수아구찜앤탕-마포본점  0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
1인용마라탕-서울수색점        0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   

UserID              -f**님  -g**님  ...  zq**님  zr**님  zs**님  zt**님  zu**님  \
Restaurant                        ...                                      
                      0.0    0.0  ...    0.0    0.0    0.0    0.0    0.0   
10평파스타&화덕피자           0.0    0.0  ...    5.0    0.0    0.0    0.0    0.0   
1인기사식당                0.0    0.0  ...    0.0    0.0    5.0    0.0    0.0   
1인살로만강명수아구찜앤탕-마포본점    0.0    0.0  ...    0.0    0.0    0.0    0.0    0.0   
1인용마라탕-서울수색

In [57]:
# 코사인 유사도 구하기
item_sim = cosine_similarity(retaurant_user, retaurant_user)

# cosine_similarity()로 반환된 넘파이 행렬을 DataFrame으로 변환
item_sim_df = pd.DataFrame(data=item_sim, index=user_resaurant.columns,
                          columns=user_resaurant.columns)

item_sim_df.head(3)

Restaurant,,10평파스타&화덕피자,1인기사식당,1인살로만강명수아구찜앤탕-마포본점,1인용마라탕-서울수색점,1인용묵은지김치찜-서울수색점,1인찌개와탕&냉면,1인한솥명가갈비찜-월곡점,1인할매보쌈족발-월곡점,24시 진짜루,...,홍콩반점-합정점,홍콩할매불닭발-당산점,홍푸드&갈비찜&냉면,홍푸드1인분&갈비찜,황가네철판곱창전골,회대장산지직송숙성활어전문점,회뜨는총각-연남점,후라이드참잘하는집-마포점,희야네반찬,힐링스시-본점
Restaurant,,,,,,,,,,,,,,,,,,,,,
,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
10평파스타&화덕피자,0.0,1.000000,0.503813,0.122297,0.268075,0.284228,0.190044,0.417337,0.216450,0.500751,...,0.369556,0.195046,0.511054,0.481359,0.503327,0.323692,0.486202,0.480500,0.300018,0.209162
1인기사식당,0.0,0.503813,1.000000,0.146155,0.217439,0.303843,0.243601,0.429522,0.222637,0.573657,...,0.355140,0.212923,0.568321,0.551610,0.573739,0.354462,0.531294,0.555018,0.305682,0.211371


In [31]:
# 예시로 '10평파스타&화덕피자' 가게와 코사일 유사도가 높은순으로 5개의 가게를 뽑아냄
item_sim_df['10평파스타&화덕피자'].sort_values(ascending=False)[:5]

Restaurant
10평파스타&화덕피자       1.000000
북촌손만두-홍익대점        0.548509
옐로우피자-연남점         0.539251
호식이두마리치킨-성산1호점    0.537769
마깐마포깐풍기-신촌본점      0.536101
Name: 10평파스타&화덕피자, dtype: float64

In [47]:
def predict_rating(ratings_arr, item_sim_arr):
    ratings_pred = ratings_arr.dot(item_sim_arr) / np.array([np.abs(item_sim_arr).sum(axis=1)]) # 평점, 가게 유사도
    return ratings_pred

# 유저별 예측 평점 구하기
# 평점 value와 유사도 value만 뽑아서 대입
ratings_pred = predict_rating(user_resaurant.values, item_sim_df.values)
ratings_pred_matrix = pd.DataFrame(data=ratings_pred, index=user_resaurant.index,
                                  columns = user_resaurant.columns)

# 유저별 계산된 예측 평점
ratings_pred_matrix

/var/folders/sf/rg3dgdy135jbcr6gtfw8f_f80000gn/T/ipykernel_8270/668443113.py:2: RuntimeWarning: invalid value encountered in true_divide
  ratings_pred = ratings_arr.dot(item_sim_arr) / np.array([np.abs(item_sim_arr).sum(axis=1)])


Restaurant,,10평파스타&화덕피자,1인기사식당,1인살로만강명수아구찜앤탕-마포본점,1인용마라탕-서울수색점,1인용묵은지김치찜-서울수색점,1인찌개와탕&냉면,1인한솥명가갈비찜-월곡점,1인할매보쌈족발-월곡점,24시 진짜루,...,홍콩반점-합정점,홍콩할매불닭발-당산점,홍푸드&갈비찜&냉면,홍푸드1인분&갈비찜,황가네철판곱창전골,회대장산지직송숙성활어전문점,회뜨는총각-연남점,후라이드참잘하는집-마포점,희야네반찬,힐링스시-본점
UserID,,,,,,,,,,,,,,,,,,,,,
,5.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
--**님,0.0,0.010059,0.009023,0.013583,0.006886,0.009273,0.009896,0.008433,0.007859,0.009457,...,0.007784,0.011707,0.009475,0.009241,0.009432,0.009210,0.009245,0.008465,0.011204,0.007393
-0**님,0.0,0.012205,0.013007,0.011258,0.010863,0.011009,0.011804,0.011672,0.012260,0.013952,...,0.012289,0.009890,0.012506,0.013311,0.013676,0.011598,0.013639,0.016790,0.011385,0.010980
-9**님,0.0,0.012417,0.014380,0.011839,0.011703,0.012901,0.011976,0.012048,0.012915,0.014144,...,0.012745,0.010879,0.014004,0.013786,0.013675,0.012844,0.013294,0.015157,0.011679,0.012945
-_**님,0.0,0.228214,0.239972,0.218854,0.207353,0.219314,0.212637,0.219047,0.225447,0.248976,...,0.228531,0.193639,0.234343,0.240288,0.245492,0.228560,0.237295,0.282069,0.212312,0.203991
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
zv**님,0.0,0.041805,0.041871,0.053234,0.041967,0.039500,0.043854,0.041123,0.042227,0.041855,...,0.040493,0.034758,0.042829,0.043632,0.042677,0.036997,0.042302,0.044192,0.041363,0.040838
zw**님,0.0,0.085870,0.101155,0.086120,0.083910,0.081277,0.084374,0.082583,0.085611,0.093168,...,0.083568,0.078466,0.088945,0.089592,0.090808,0.083820,0.088873,0.099484,0.078562,0.079500
zx**님,0.0,0.990380,1.057000,0.974466,0.970112,0.954100,0.943469,0.997160,1.010494,1.080132,...,0.994220,0.902532,1.040052,1.038007,1.067007,0.960304,1.032068,1.164856,0.953016,0.919340


In [58]:
# 위에서 나온 유저-가게별 평점 예측표를 기반으로 mse를 통해 예측 정확성을 분석함
# mse가 높기때문에 그다지 좋지 않음
def get_mse(pred, actual):
    # 실제 평점이 있는 가게만 추출
    pred = pred[actual.nonzero()].flatten()
    actual = actual[actual.nonzero()].flatten()
    return mean_squared_error(pred, actual)
print('MSE: ', get_mse(ratings_pred, user_resaurant.values))

MSE:  10.422781104905408


In [64]:
# 아래의 username에 유저 이름을 넣어 해당 유저에게 추천하는 식당 반환
username='ds**님'

# 사용자의 예측 평점이 높은 가게 중, 사용자가 실제로 평점을 메기지 않은(아직 안먹어본) 가게를 반환
def non_visit_restaurant(ratings_matrix, username):
    # username로 입력받은 사용자의 모든 식당 정보를 추출해 Series로 반환
    # 반환된 user_rating은 Restaurant를 인덱스로 가지는 Series 객체
    user_rating = ratings_matrix.loc[username, :]

    # user_rating이 0보다 크면 기존에 방문한 식당
    # 대상 인덱스를 추출해 list 객체로 만듦
    visited = user_rating[user_rating>0].index.tolist()

    # 모든 식당을 list 객체로
    restaurant_list = ratings_matrix.columns.tolist()

    # list comprehension으로 visited에 해당하는 식당은 restaurant_list에서 제외
    not_visited = [restaurant for restaurant in restaurant_list if restaurant not in visited]

    return not_visited

# 예측 평점 DataFrame에서 사용자 id 인덱스와 not_visited에 들어온 식당 뽑기
# 가장 예측 평점이 높은 순으로 정렬
def recomm_restaurant_by_username(pred_df, userId, not_tried, top_n):
    recomm_restaurant = pred_df.loc[userId, not_tried].sort_values(ascending=False)[:top_n]
    return recomm_restaurant



# 예측 평점 데이터는 DataFrame으로 
ratings_pred_matrix = pd.DataFrame(data=ratings_pred, index=user_resaurant.index,
                                   columns=user_resaurant.columns)

# 유저가 방문하지 않은 식당 추출
not_tried = non_visit_restaurant(user_resaurant, username)

# 상위 3개의 식당 추천
recomm_restaurant = recomm_restaurant_by_username(ratings_pred_matrix, username, not_tried, top_n=3)
recomm_restaurant = pd.DataFrame(data=recomm_restaurant.values, index=recomm_restaurant.index,
                           columns=['predict rate'])
recomm_restaurant

,predict rate
Restaurant,
국내산1등고기삼육공,1.125728
명품임실치즈피자-이대점,1.047182
마포구회맛집,1.035069
